<span style='font-size: 50px'>Projeto Final</span>

**Disciplina:** Projeto Nível II em Controle e Processamento de Sinais I

**Aluno:** Bruno Mingoti

**Professor:** Joceli Mayer


# Importações

In [1]:
import cv2
import math
import pyautogui as pag # para comandos ao computador (mouse, teclado, etc)
import numpy as np
import serial # para comunicação serial com o arduino

# Inicializações

In [2]:
button_pressed = False
button_counter = 0
button_delay = 25
gesture_threshold = 300
mode_delay = 25
mode = None
mode_selected = False
mode_counter = 0
fps_counter = 0
waiting_time = 75

In [3]:
id_camera = 2 # alterar para outro número caso esteja sendo utilizada uma câmera externa

In [4]:
# Configurações da porta serial
try:
    config_level = '0'
    portaSerial = serial.Serial('/dev/ttyACM0', 9600) # a porta pode mudar de acordo com o computador. Verifique no gerenciador de dispositivos
    portaSerial.timeout = 1
    serial_port = True
except:
    print('Não foi possível conectar à porta serial.')
    serial_port = False
    pass

# É importate ressaltar que a execução só será possível se houver uma conexão com a porta serial compatível.

Não foi possível conectar à porta serial.


# Segmentação

In [5]:
class Hand:

    def __init__(self, binary, masked, raw, frame):
        self.masked = masked
        self.binary = binary
        self._raw = raw
        self.frame = frame
        self.contours = []
        self.outline = self.draw_outline()
        self.fingertips = self.extract_fingerpoints()


    def draw_outline(self, min_area=10000, color=(0, 255, 0), thickness=2):
        '''
        Desenha o contorno da mão e retorna a imagem.
        params:
            min_area: área mínima da mão para ser considerada
            color: cor do contorno
            thickness: espessura do contorno

        return:
            self.frame: imagem com o contorno desenhado
        '''

        # Encontra os contornos da imagem binária, ou seja, a curva que une todos os pontos de mesma cor ou intensidade
        # self.binary: imagem binária
        # cv2.RETR_TREE: modo de recuperação de contorno. Cada controno é armazenado em forma de um vetor de pontos
        # cv2.CHAIN_APPROX_SIMPLE: modo de aproximação de contorno
        contours, _ = cv2.findContours(
            self.binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        palm_area = 0
        flag = None
        cnt = None

        for (i, c) in enumerate(contours):
            # Calcula a área de contorno, utilizando a fórmula de Green.
            area = cv2.contourArea(c)
            if area > palm_area:
                palm_area = area
                flag = i
        if flag is not None and palm_area > min_area:
            cnt = contours[flag]
            self.contours = cnt
            cpy = self.frame.copy()
            # Desenha o contorno da mão
            cv2.drawContours(cpy, [cnt], 0, color, thickness)
            return cpy
        else:
            return self.frame


    def extract_fingerpoints(self):
        '''
        Extrai os pontos entre os dedos da mão e executa todos os comandos com base na quantidade na quantidade de defeitos de convexidade retornados.

        return:
            Lista contendo os pontos entre os dedos da mão.
        '''
        global button_pressed, button_counter, button_delay, config_level, mode, mode_delay, mode_selected, mode_counter, fps_counter, waiting_time
        
        cnt = self.contours
        if len(cnt) == 0:
            return cnt
        points = []

        # Encontra o convex hull (envoltório convexo, em tradução literal) de um conjunto de pontos 2D usando o algoritmo de Sklansky.
        hull = cv2.convexHull(cnt, returnPoints=False) # Parâmetros: contour, hull, returnPoints (se True, retorna os pontos do contorno. Se False, retorna os índices dos pontos do contorno)

        # Encontra os defeitos de convexidade de um contorno.
        defects = cv2.convexityDefects(cnt, hull) # Parâmetros: contour, convexhull (contém os índices dos pontos do contorno)
        count_defects = 0
        # O for loop itera por cada linha do array de defects, que representa um convexity defect.
        for i in range(defects.shape[0]):
            # Esses índices representam o início, o fim e o ponto distante (indica o ponto de profundidade máxima do defeito de convexidade) e a distância entre o far point e o convex hull.
            s, e, f, d = defects[i, 0]
            # Os índices s, e e f são usados para acessar os pontos correspondentes na matriz cnt, que contém os contornos das mãos. Esses pontos são convertidos em tuplas, representando as coordenadas (x, y) dos pontos inicial, final e distante do defeito de convexidade.
            start = tuple(cnt[s][0])
            end = tuple(cnt[e][0])
            far = tuple(cnt[f][0])
        
        # Lei dos cossenos: calcula o ângulo entre 3 pontos.
        # a, b, c: lados do triângulo, representados por start, end e far, respectivamente.
            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

            # Se o ângulo entre os dedos for menor que 80 graus, então é considerado um defeito de convexidade e o contador é incrementado (para posterior cv2.Circle)
            if angle <= 80:
                count_defects += 1
                points.append(far)

        
        # Seleção do modo de operação: video player control ou led control
        # Foi adicionado um delay para evitar que o modo seja selecionado acidentalmente
        # Também foi adicionado um delay entre o pressionamento dos botões pelo PyAutoGUI para evitar que o mesmo botão seja pressionado várias vezes.
        if mode is None:
            if mode_selected == False:
                if count_defects == 2:
                    if waiting_time > 125:
                        mode = 'player_control'
                        mode_selected = True
                        waiting_time = 0
                elif count_defects == 3 and serial_port:
                    if waiting_time > 125:
                        mode = 'led_control'
                        mode_selected = True
                        waiting_time = 0
        else:
            if button_pressed == False:
                if mode == 'player_control':
                    if count_defects == 0:
                        pag.press("space")
                        button_pressed = True
                        fps_counter = 0
                    elif count_defects == 1:
                        pag.press("right")
                        button_pressed = True
                        fps_counter = 0
                    elif count_defects == 2:
                        pag.press("left")
                        button_pressed = True
                        fps_counter = 0
                    elif count_defects == 3: 
                        pag.press("up")
                        button_pressed = True 
                        fps_counter = 0
                    elif count_defects == 4:
                        pag.press("down")
                        button_pressed = True
                        fps_counter = 0
                    else:
                        pass

                elif serial_port:
                    if mode == 'led_control': # alterado
                        if count_defects == 0:  
                            config_level = '0'
                            mode_selected = True
                            fps_counter = 0
                        elif count_defects == 1:
                            config_level = '1'
                            mode_selected = True
                            fps_counter = 0
                        elif count_defects == 2:  
                            config_level = '2'
                            mode_selected = True
                            fps_counter = 0
                        elif count_defects == 3:
                            config_level = '3'
                            mode_selected = True
                            fps_counter = 0
                        elif count_defects == 4:
                            config_level = '4'
                            mode_selected = True
                            fps_counter = 0
                        else:
                            pass
                        # Envio de dados para a porta serial
                        portaSerial.write(config_level.encode())


        # blocos de código para delay para evitar que o mesmo botão seja pressionado várias vezes
        if button_pressed:
            button_counter += 1
            if button_counter > button_delay:
                button_counter = 0
                button_pressed = False

        if mode_selected:
            mode_counter += 1
            if mode_counter > mode_delay:
                mode_counter = 0 
                mode_selected = False 
        else:
            if count_defects == 2 or count_defects == 3:
                waiting_time += 1
            else:
                waiting_time = 0
            

        return [pt for idx, pt in zip(range(5), points)]


    def get_center_of_mass(self):
        '''
        Obtém o centro de massa da mão.

        return:
            Tupla contendo as coordenadas (x, y) do centro de massa da mão.
        '''
        if len(self.contours) == 0:
            return None
        # função moments: calcula momentos até a 3 ordem de froam vetorial ou rasterizada.
        M = cv2.moments(self.contours) # M: dicionário com os momentos da imagem
        cX = int(M["m10"] / M["m00"]) # Posição x do centro de massa
        cY = int(M["m01"] / M["m00"]) # Posição y do centro de massa
        return (cX, cY)

In [6]:
def show_image(frame, window_name='frame'):
    '''
    Exibe a imagem em uma janela.
    '''
    
    while True:
        cv2.imshow(window_name, frame)
        key = cv2.waitKey(10)
        if key == 27:
            cv2.destroyAllWindows()
            break

In [7]:
def region_selection(camera_id, out_width, out_height, num_positions=3):
    '''
    Obtém a média dos histogramas de cor de uma região da mão.

    params:
        camera_id: id da câmera
        out_width: largura da imagem de saída
        out_height: altura da imagem de saída
        num_positions: número de posições diferentes para a mão

    return: 
        cap: objeto VideoCapture
        histograms: lista contendo os histogramas de cor de cada posição
        object_hist: histograma de cor da região selecionada
    '''

    cap = cv2.VideoCapture(camera_id)
    
    histograms = []
    
    for _ in range(num_positions):
        while True:
            _, frame = cap.read()
            frame = cv2.flip(frame, 1)
            frame = cv2.resize(frame, (out_width, out_height))
            cv2.rectangle(frame, (500, 100), (580, 180), (255, 0, 0), 2)
            cv2.putText(frame, "Configuration: Place region of the hand inside blue box and press 'S' key 3 times, changing the angle of the hand.", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            box = frame[105:175, 505:575]
            cv2.imshow("Capture Histogram", frame)
            key = cv2.waitKey(10)
            
            if key == ord('s'):
                object_color = box
                hsv_object_color = cv2.cvtColor(object_color, cv2.COLOR_BGR2HSV) # conversão para HSV
                object_hist = cv2.calcHist([hsv_object_color], [0, 1], None, [12, 15], [0, 180, 0, 256]) # Retorna o histograma de cores da região selecionada
                cv2.normalize(object_hist, object_hist, 0, 255, cv2.NORM_MINMAX) # normaliza o histograma para valores entre 0 e 255
                histograms.append(object_hist)
                cv2.destroyAllWindows()
                break
            
            if key == 27:
                cv2.destroyAllWindows()
                cap.release()
                return None, None  # Retorna None se o usuário abortar a seleção pressionando a tecla q
    
    return cap, histograms, object_hist

In [8]:
def object_localization(frame, object_hist):
    '''
    Realiza a segmentação da mão na imagem.
    
    params:
        frame: imagem
        object_hist: histograma de cor da região selecionada

    return:
        closing: imagem binária
        masked: imagem com a mão segmentada
        segment_thresh: imagem binária
    ''' 
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # converte para HSV
    # show_image(hsv_frame)
    object_segment = cv2.calcBackProject([hsv_frame], [0, 1], object_hist, [0, 180, 0, 256], 1) # Cálculo do modelo de histograma de uma determinada features da imagem, nesse caso, o tom de pele
    # show_image(object_segment) 
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9)) 
    # show_image(disc)
    cv2.filter2D(object_segment, -1, disc, object_segment)
    # show_image(object_segment)
    _, segment_thresh = cv2.threshold(object_segment, 70, 255, cv2.THRESH_BINARY)
    # show_image(segment_thresh)
    eroded = cv2.erode(segment_thresh, None, iterations=2)
    # show_image(eroded)
    dilated = cv2.dilate(eroded, None, iterations=2)
    # show_image(dilated)
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, None)
    # show_image(closing)
    masked = cv2.bitwise_and(frame, frame, mask=closing)
    # show_image(masked) # limitar a fundo totalmente branco pode ser uma medida de contorno
    return closing, masked, segment_thresh

In [9]:
# Captura os histograma 3 vezes, solicitando ao usuário o posicionamento da mão em posições diferentes
cap, histograms, object_hist = region_selection(id_camera, 1000, 600, num_positions=3) # Alterar índice da câmera, caso ncessário

# Cálculo da média dos histogramas
if histograms:
    object_color = np.mean(histograms, axis=0)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/neo/Documentos/bmt/graduacao/codes/projeto_controle_1/venv/lib/python3.10/site-packages/cv2/qt/plugins"


# Main

In [ ]:
while True:
    # captura o frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # captura a região de interesse para a segmentação
    region_coordinates = (100, 100, 250, 250)
    x, y, w, h = region_coordinates
    roi = frame[y:y+h, x:x+w]
    detected_hand, masked, raw = object_localization(roi, object_hist)
    fps_counter += 1

    if fps_counter == 750:
        fps_counter = 0
        mode_selected = False
        mode = None

    
    hand = Hand(detected_hand, masked, raw, roi)
    custom_outline = hand.draw_outline(
        min_area=10000, color=(0, 255, 255), thickness=2)

    quick_outline = hand.outline

    for fingertip in hand.fingertips:
        cv2.circle(quick_outline, fingertip, 5, (0, 0, 255), -1)

    # Controle dos LEDs
    if mode == 'led_control' and serial_port:
        if len(hand.fingertips) == 0:
            cv2.putText(quick_outline, "Off", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        elif len(hand.fingertips) == 1:
            cv2.putText(quick_outline, "Config 1", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        elif len(hand.fingertips) == 2:
            cv2.putText(quick_outline, "Config 2", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        elif len(hand.fingertips) == 3:
            cv2.putText(quick_outline, "Config 3", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        elif len(hand.fingertips) == 4:
            cv2.putText(quick_outline, "Config 4", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        else:
            pass

    # Controle da TV e player de vídeo
    elif mode == 'player_control':
        if len(hand.fingertips) == 0:
            cv2.putText(quick_outline, "Play/Pause", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        elif len(hand.fingertips) == 1:
            cv2.putText(quick_outline, "Forward", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        elif len(hand.fingertips) == 2:
            cv2.putText(quick_outline, "Backward", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        elif len(hand.fingertips) == 3:
            cv2.putText(quick_outline, "Volume UP", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        elif len(hand.fingertips) == 4:
            cv2.putText(quick_outline, "Volume Down", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,255), 2)
        else:
            pass
    
    elif mode == None:
        text = "Select mode\n- 3 fingers:\n    video player control\n- 4 fingers:\n    led control"
        y0, dy = 30, 20
        for i, line in enumerate(text.split('\n')):
            y = y0 + i*dy
            cv2.putText(quick_outline, line, (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    else:
        pass


    com = hand.get_center_of_mass()
    if com:
        cv2.circle(quick_outline, com, 10, (255, 0, 0), -1)

    cv2.imshow("Hand Control", quick_outline)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
if serial_port:
    portaSerial = serial.Serial.close(portaSerial)

# Comentários

O projeto final consistiu em uma implementação de um sistema de controle de TV e de luminosidade por meio da segmentação das mãos e reconhecimento de gestos.

Primeiramente, com relação à segmentação, seguiu-se o método de captura do histograma da cor da pele e posterior conversão para o sistema HSV e aplicações das técnicas de segmentação.

Após esse processo, os seguintes passos foram seguidos:
- Desenho do contorno da mão;
- Aplicação do convex hull, envoltório que passa por todas as extremidades do contorno feito na etapa anterior;
- Localização os pontos de defeitos de convexidades entre o envoltório e o contorno;
- Seleção do ponto mais profundo de cada defeito de convexidade, correspondente ao ponto entre os dedos;
- Aplicação da Lei dos Cossenos a fim de encontrar o ângulo;
- Estipulação de um sistema de contagem de defeitos de convexidade mediante ângulo aplicado entre cada um dos dedos;


Uma vez localizados todos os pontos de interesse e a contagem desses pontos, desenvolveu-se um sistema de seleção do modo de operação a ser executado. Vale salientar que a seleção ocorre 125 frames de delay (a fim de evitar erros do usuário):
- 3 dedos (2 pontos de convexidade): Controle de TV;
- 4 dedos (3 pontos de convexidade): Controle de luminosidade, via porta serial (se disponível).

Após um certo tempo de inatividade o programa retorna ao modo de seleção de operação.


Os seguintes comandos foram implementados em cada um dos casos de contagem dos pontos de convexidade:
CONTROLE DE TV:
- 0: Play/Pause
- 1: Forward
- 2: Backward
- 3: Volume Up
- 4: Volume Down

CONTROLE DE LUMINOSIDADE
- 0: Off
- 1: Configuração 1
- 2: Configuração 2
- 3: Configuração 3
- 4: Configuração 4

Cada nível de configuração corresponde a um nível de luminosidade, do mais baixo para o mais alto.
